In [10]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv

from utils.knn import KNN
from utils.helper import HELPER

In [54]:
from imblearn.under_sampling import CondensedNearestNeighbour

ImportError: cannot import name '_check_sample_weight' from 'sklearn.utils.validation' (/home/ajay_chinni/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py)

In [2]:
hl = HELPER()

#only do once to convert .data to .csv file change param to True to run it
hl.get_data(do_it = False)

False


In [3]:
df = pd.read_csv('data/ecoli.csv')
# data has 8 features
print(df.columns)
print("shape of dataframe ",df.shape)

Index(['Sequence_Name', 'mcg', 'gvh', 'lip', 'chg', 'aac', 'alm1', 'alm2',
       'label'],
      dtype='object')
shape of dataframe  (336, 9)


In [4]:
# 8 classes the last three has very few data points so we can remove those classes 
df.label.value_counts()

cp     143
im      77
pp      52
imU     35
om      20
omL      5
imL      2
imS      2
Name: label, dtype: int64

In [5]:
# removed the last 3 clases
df = df[~(df['label'].isin(["imL","imS","omL"]))]

#choose one_hot_encode or label_encode
df = hl.encoding(df,encode='one_hot_encode')
print("new shape after encoding is ",df.shape)
df.to_csv("dummy.csv",index = False)

#converting string categorical values to integer for label
le1 = LabelEncoder()
df['label'] = le1.fit_transform(df['label'])
df.label.value_counts()

new shape after encoding is  (327, 335)


0    143
1     77
4     52
2     35
3     20
Name: label, dtype: int64

In [7]:
#split data into train test
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,df.columns !='label'].values,df.label.values, test_size=0.2,shuffle = True)

#Intilizing knn classifier with 3 neighbours
clf_model = KNN(k = 3)
clf_model.fit(X_train,y_train)
predictions = clf_model.predict(X_test)
acc = clf_model.accuracy(predictions,y_test)
print("Accuracy is",round(acc,2)*100,"%")
print(confusion_matrix(predictions,y_test))

Accuracy is 86.0 %
[[26  1  0  0  1]
 [ 0 15  2  0  0]
 [ 0  4  3  0  0]
 [ 0  0  0  3  1]
 [ 0  0  0  0 10]]


In [51]:
#cross validation statified k fold 
skf = StratifiedKFold(n_splits = 5)
x = df.iloc[:,df.columns !='label'].values
y = df.label.values

acc_lst = []
for train_indx, test_indx in skf.split(x,y):
    x_Train,x_test = x[train_indx],x[train_indx]
    y_Train,y_test = y[train_indx],y[train_indx]
    #check the value counts for every iteration
#     unique, counts = np.unique(y_Train, return_counts = True)
#     print(np.asarray((unique,counts)).T)
    
    clf_model = KNN(3)
    clf_model.fit(x_Train,y_Train)
    pred=clf_model.predict(x_test)
    acc = clf_model.accuracy(pred,y_test)
    acc_lst.append(acc)
    print("Accuracy is",round(acc,2)*100,"%")
#     print(confusion_matrix(pred,y_test))
    print("--------------------")
print("mean accuracy ",np.array(acc_lst).mean())

Accuracy is 94.0 %
--------------------
Accuracy is 92.0 %
--------------------
Accuracy is 93.0 %
--------------------
Accuracy is 94.0 %
--------------------
Accuracy is 94.0 %
--------------------
mean accuracy  0.9334809517536711


Observations

1.if the categorical variable is label encoded than we get accuracy around 50 % if we remove the column we get accuaracy around 90% but if we really want to use the feature we can one hot encode it and we get 85% accuracy but suprisingly all the ~390 rows were unique so due to it. One hot encoding created 390 extra column this explotion should have lead to curse of dimentiality and bring down the accuracy.